In [1]:
import pandas as pd
import numpy as np
from keras import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from tqdm import tqdm

Using TensorFlow backend.
/Users/ambroiserenaud/anaconda3/envs/chili/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/ambroiserenaud/anaconda3/envs/chili/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/ambroiserenaud/anaconda3/envs/chili/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

## Data preparation

In [145]:

data = pd.read_csv('induce-data-2019-08-08.csv').iloc[:, :]
vocab = ['C_E_F_T',
         'C_E_F_C',
         'C_E_F_O',
         'A_E_F_T',
         'A_E_F_O',
         'A_E_F_C',
         'G_E_F_C',
         'G_E_F_T',
         'G_E_F_O',
         'A_E_M_T',
         'A_E_M_O',
         'A_E_M_C',
         'G_E_M_O',
         'G_E_M_C',
         'G_E_M_T',
         'C_E_M_O',
         'C_E_M_C',
         'C_E_M_T',
         'C_H_F_CO',
         'C_H_F_CT',
         'C_H_F_OT',
         'G_H_F_OT',
         'G_H_F_CO',
         'G_H_F_CT',
         'A_H_F_CT',
         'A_H_F_OT',
         'A_H_F_CO',
         'C_H_M_CO',
         'C_H_M_CT',
         'C_H_M_OT',
         'A_H_M_CT',
         'A_H_M_OT',
         'A_H_M_CO',
         'G_H_M_OT',
         'G_H_M_CO',
         'G_H_M_CT', ]

labels = ['correct',
          'wrong',
          'type',
          'orientation',
          'color']

types = ['INTRO',
         'CORE',
         'FLEX',
         'TRIK',
         'DELY'
]

topics = ['cards',
          'animals',
          'geometry'
    
]

feat = ['type',
        'color',
        'orientation',
        'dual'
]

age = ['8-10','11-13']

# qts_id = ['A_E_F_C_INTRO', 'A_E_F_O_INTRO', 'A_E_F_T_INTRO', 'A_E_M_C_INTRO',
#         'A_E_M_O_INTRO', 'A_E_M_T_INTRO', 'A_H_F_CO_CORE', 'A_H_F_CO_DELY',
#         'A_H_F_CO_TRIK', 'A_H_F_CT_CORE', 'A_H_F_CT_DELY', 'A_H_F_CT_TRIK',
#         'A_H_F_OT_CORE', 'A_H_F_OT_DELY', 'A_H_F_OT_TRIK', 'A_H_M_CO_CORE',
#         'A_H_M_CO_FLEX', 'A_H_M_CT_CORE', 'A_H_M_CT_FLEX', 'A_H_M_OT_CORE',
#         'A_H_M_OT_FLEX', 'C_E_F_C_INTRO', 'C_E_F_O_INTRO', 'C_E_F_T_INTRO',
#         'C_E_M_C_INTRO', 'C_E_M_O_INTRO', 'C_E_M_T_INTRO', 'C_H_F_CO_CORE',
#         'C_H_F_CO_DELY', 'C_H_F_CO_TRIK', 'C_H_F_CT_CORE', 'C_H_F_CT_DELY',
#         'C_H_F_CT_TRIK', 'C_H_F_OT_CORE', 'C_H_F_OT_DELY', 'C_H_F_OT_TRIK',
#         'C_H_M_CO_CORE', 'C_H_M_CO_FLEX', 'C_H_M_CT_CORE', 'C_H_M_CT_FLEX',
#         'C_H_M_OT_CORE', 'C_H_M_OT_FLEX', 'G_E_F_C_INTRO', 'G_E_F_O_INTRO',
#         'G_E_F_T_INTRO', 'G_E_M_C_INTRO', 'G_E_M_O_INTRO', 'G_E_M_T_INTRO',
#         'G_H_F_CO_CORE', 'G_H_F_CO_DELY', 'G_H_F_CO_TRIK', 'G_H_F_CT_CORE',
#         'G_H_F_CT_DELY', 'G_H_F_CT_TRIK', 'G_H_F_OT_CORE', 'G_H_F_OT_DELY',
#         'G_H_F_OT_TRIK', 'G_H_M_CO_CORE', 'G_H_M_CO_FLEX', 'G_H_M_CT_CORE',
#         'G_H_M_CT_FLEX', 'G_H_M_OT_CORE', 'G_H_M_OT_FLEX']




def seq_to_int(qts,vocab, labels, types, feat, topics, n_steps, age):
    integ = list()
    for i,x in enumerate(qts):
        if i != n_steps:
            features = list()
            features.append(vocab.index(qts[i, 4]))
            features.append(labels.index(qts[i, 5]))
            features.append(qts[i, 2])
            features.append(types.index(qts[i, 9]))
            features.append(feat.index(qts[i, 10]))
            features.append(topics.index(qts[i, 6]))
            features.append(age.index(qts[i, 7]))
        else:
            features = list()
            features.append(vocab.index(qts[i, 4]))
            features.append(-1)
            features.append(qts[i, 2])
            features.append(types.index(qts[i, 9]))
            features.append(feat.index(qts[i, 10]))
            features.append(topics.index(qts[i, 6]))
            features.append(age.index(qts[i, 7]))
        integ.append(features)
    return integ

def split_sequence(data, n_steps, vocab, labels, types, feat, topics, age):
    X, Y = list(), list()
    users = list(dict.fromkeys(data.loc[:, "user"]))
    for u in users:
        sequence = data[data.user == u]
        for i in range(len(sequence)):
            end_idx = i + n_steps
            if end_idx > len(sequence)-1:
                break
            x = seq_to_int(sequence.iloc[i:end_idx+1, :].values, vocab, labels, types, feat, topics, n_steps, age)
            y = labels.index(str(sequence.iloc[end_idx, 5]))
            X.append(x)
            Y.append(y)
    return np.array(X), np.array(Y)


### Test split_sequence


In [147]:
print(split_sequence(data.iloc[:20, :], 10, vocab, labels, types, feat, topics, age)[0].shape)


(10, 11, 7)


## The model


In [177]:

data_train, data_test = np.split(data.sample(frac=1, random_state=42), 
                                   [int(0.8 * len(data))])
print(len(data_train),len(data_test))


11173 2794


In [233]:

# n_steps = 10

X_train_split = list()
y_train_split = list()


for i in tqdm(range(5, 10)):
    X_seq, y_seq = split_sequence(data_train, i, vocab, labels, types, feat, topics, age)
    for x in X_seq:
        X_train_split.append(x)
    for _y in y_seq:
        y_train_split.append(_y)
y_train = np_utils.to_categorical(y_train_split)
# X = np_utils.to_categorical(X)
X_train = np.array(X_train_split)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
X_test_split = list()
y_test_split = list()


for i in tqdm(range(5, 10)):
    X_seq, y_seq = split_sequence(data_test, i, vocab, labels, types, feat, topics, age)
    for x in X_seq:
        X_test_split.append(x)
    for _y in y_seq:
        y_test_split.append(_y)
y_test = np_utils.to_categorical(y_test_split)
# X = np_utils.to_categorical(X)
X_test = np.array(X_test_split)




100%|██████████| 5/5 [00:24<00:00,  4.88s/it]

100%|██████████| 5/5 [00:03<00:00,  1.28it/s]


In [221]:
n_features = 7
# X = X.reshape((X.shape[0], X.shape[1], n_features*X.shape[-1]))
# print(X.shape)


print('train set size: '+str(len(y_train)))
print('test set size: '+str(len(y_test)))

model = Sequential()
model.add(LSTM(256, input_shape=(None, n_features)))
# model.add(Dropout(0.2))
# model.add(LSTM(X.shape[1], return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(X.shape[1]))
# model.add(Dropout(0.2))
amsgrad = Adam(amsgrad=True)
model.add(Dense(5))
model.compile(optimizer=amsgrad, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

train set size: 48095
test set size: 6288


### Training

In [246]:
def train_generator():
    while True:
        idx = np.random.choice(np.arange(len(y_train)), 30, replace=True)
        x_train = np.take(X_train,idx)
        _y_train = np.take(y_train,idx)
        yield np.array([x_train]), np.array([_y_train])
#model.fit(X_train, y_train, epochs=1000, batch_size=300)
model.fit_generator(train_generator(), steps_per_epoch=X_train.shape[0], epochs=2000, verbose=1)

(48095,)


ValueError: Error when checking input: expected lstm_10_input to have 3 dimensions, but got array with shape (48095, 1)

### Validation

In [59]:
def test_generator(batch_size=1):
    while True:
        idx = np.random.choice(np.arange(len(y_test)), batch_size, replace=False)[0]
        x_test np.take(X_test, idx)
        _y_test = np.take(y_test)
        _y_test = y_test[idx]
        yield np.array([x_test]), np.array([_y_test])
scores = model.evaluate_generator(test_generator(), 1000)

def test_18_generator():
    while True:
        idx = np.random.choice(np.arange(len(y_test)), 1, replace=False)[0]
        x_test = X_test[idx]
        _ytest = y_test[idx]
        while _ytest[0] == 1 or _ytest[1]==1:
            idx = np.random.choice(np.arange(len(y_test)), 1, replace=False)[0]
            x_test = X_test[idx]
            _ytest = y_test[idx]
        yield np.array([x_test]), np.array([_ytest])
        
def test_inf_18_generator():
    while True:
        idx = np.random.choice(np.arange(len(y_test)), 1, replace=False)[0]
        x_test = X_test[idx]
        _ytest = y_test[idx]
        while _ytest[0] != 1 and _ytest[1]!=1:
            idx = np.random.choice(np.arange(len(y_test)), 1, replace=False)[0]
            x_test = X_test[idx]
            _ytest = y_test[idx]
        yield np.array([x_test]), np.array([_ytest])





In [60]:
#model.save("model.h5")
#from keras.models import load_model
#model = load_model("model.h5")

In [61]:
scores = model.evaluate_generator(test_generator(), 1000)
print("Accuracy: %.2f%%" % (scores[1]*100))

print(X_test[0])
print(model.predict(np.array([X_test[0]])))
print(np.array([X_test[1]]))
print(y_test[0])

Accuracy: 53.90%
[[26  4 58  3  3  1  1]
 [ 8  0  8  0  2  2  1]
 [24  4 44  4  3  1  1]
 [ 3  0  0  0  0  1  1]
 [24  2 21  1  3  1  1]
 [21 -1 24  1  3  2  1]]
[[-0.0896569   0.07918497  0.20062298 -0.06545986  0.88510835]]
[[[ 8  0  8  0  2  2  1]
  [24  4 44  4  3  1  1]
  [ 3  0  0  0  0  1  1]
  [24  2 21  1  3  1  1]
  [21  2 24  1  3  2  1]
  [25 -1 51  4  3  1  1]]]
[0. 0. 1. 0. 0.]


#### Questions greater than 18

In [62]:
scores = model.evaluate_generator(test_18_generator(), 1000)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 44.30%


#### Questions lower than 18

In [63]:
scores = model.evaluate_generator(test_inf_18_generator(), 1000)
print("Accuracy: %.2f%%" % (scores[1]*100))



Accuracy: 86.40%
